In [1]:
import os
import sys
import codecs
import collections

In [2]:
import nltk
from lxml import etree
from lxml.html.clean import Cleaner
from pymystem3 import Mystem

In [3]:
file = codecs.open('datasets/golos_io_topics.xml', encoding='utf-8')
data = file.read();

In [4]:
cleaner = Cleaner()
cleaner.allow_tags = ['item', 'field']
cleaner.remove_unknown_tags = False
clean_data = cleaner.clean_html(data)

In [6]:
fout = codecs.open('datasets/golos_io_topics-res.xml', encoding='utf-8', mode='w+')
fout.write(clean_data)
fout.close()

In [7]:
# формируем список стоп-слов и добавляем свои
from nltk.corpus import stopwords
stopwords_list = stopwords.words('russian')
more_stopwords = 'https, http, без, более, бы, был, была, были, было, быть, вам, вас, ведь, весь, вдоль, вместо, вне, вниз, внизу, внутри, во, вокруг, вот, все, всегда, всего, всех, вы, где, да, давай, давать, даже, для, до, достаточно, его, ее, её, если, есть, ещё, же, за, здесь, из, из-за, или, им, иметь, их, как, как-то, кто, когда, кроме, кто, ли, либо, мне, может, мои, мой, мы, на, навсегда, над, надо, наш, не, него, неё, нет, ни, них, но, ну, об, однако, он, она, они, оно, от, отчего, очень, по, под, после, потому, потому что, почти, при, про, снова, со, так, также, такие, такой, там, те, тем, то, того, тоже, той, только, том, тут, ты, уже, хотя, чего, чего-то, чей, чем, что, чтобы, чьё, чья, эта, эти, это'
more_stopwords_list = more_stopwords.split(", ")
stopwords_list += more_stopwords_list

In [8]:
tree = etree.fromstring(clean_data)
items = tree.xpath("//field[@name='body']")

In [9]:
seq_list = [item.text for item in items]

In [10]:
fout = codecs.open('datasets/golos_io_topics-res2.txt', encoding='utf-8', mode='w+')
for item in seq_list:
    print(item, file=fout, sep="\n")
fout.close()

In [11]:
mystem = Mystem()

def convert_prefix2utags(mystem_prefix):
    utags_dict = {'_A':       '_ADJ',
                  '_ADV':     '_ADV',                                                                                                                                                                                                                                                                    
                  '_ADVPRO':  '_ADV',                                                                                                                                                                                                                                                                    
                  '_ANUM':    '_ADJ',                                                                                                                                                                                                                                                                 
                  '_APRO':    '_DET',                                                                                                                                                                                                                                                                   
                  '_COM':     '_ADJ',                                                                                                                                                                                                                                                                    
                  '_CONJ':    '_SCONJ',                                                                                                                                                                                                                                                                  
                  '_INTJ':    '_INTJ',                                                                                                                                                                                                                                                                   
                  '_NONLEX':  '_X',                                                                                                                                                                                                                                                                      
                  '_NUM':     '_NUM',                                                                                                                                                                                                                                                                    
                  '_PART':    '_PART',                                                                                                                                                                                                                                                                   
                  '_PR':      '_ADP',                                                                                                                                                                                                                                                                    
                  '_S':       '_NOUN',                                                                                                                                                                                                                                                                   
                  '_SPRO':    '_PRON',                                                                                                                                                                                                                                                                   
                  '_UNKN':    '_X',                                                                                                                                                                                                                                                                      
                  '_V':       '_VERB',
                  '':        ''}
    return utags_dict[mystem_prefix]

def get_word_prefix(word):
    lemmas = mystem.analyze(word)
    prefix = ""
    if (isinstance(lemmas, collections.Iterable)) and ('analysis' in lemmas[0]):
        try:
            prefix = "_"+(lemmas[0]['analysis'][0]['gr'].split("=")[0].split(",")[0]) 
        except IndexError:
            prefix = ""
    return convert_prefix2utags(prefix)

def lemmatize_words(word_list):
    processed_word_list = []
    for word in word_list:
        word = mystem.lemmatize(word)[0]
# remove lemma prefixes        
#        word_w_prefix = word+get_word_prefix(word)
        processed_word_list.append(word)
    return processed_word_list    

In [12]:
print(lemmatize_words(["люди"]))

['человек']


In [13]:
def remove_stopwords(word_list):
        processed_word_list = []
        
        
        for word in word_list:
            word = word.lower() # in case they arenet all lower cased
            if word not in stopwords_list and len(word) > 2:
                processed_word_list.append(word)
        return processed_word_list    

In [14]:
# print (stopwords_list)

In [15]:
test_tokenized = nltk.word_tokenize(seq_list[1100])
test_clean = remove_stopwords(test_tokenized)

In [16]:
print(test_clean)

['//s15.postimg.org/dv7fam3ln/a2110ac82294.jpg', 'всем', 'привет', 'недавно', 'одна', 'девочка', 'звала', 'crp.center', 'освободилась', '1200', 'долларов', 'думаю', 'эххх', '...', 'вложила', 'криптовалюту', 'верила', 'проект', 'знаю', 'почему', 'слава', 'богу', 'интуиция', 'подвела', 'говорит', '1000', 'долларов', 'сняла', 'туда', '000', 'слава', 'богу', 'представьте', 'сколько', 'людей', 'потеряло', 'печально', 'видео', 'обращение', 'виталий', 'говорит', 'будут', 'работать', 'обычно', 'сказки', 'вкладывайте', '//www.youtube.com/watch', 'time_continue=393', 'v=g9v3ix73moy', 'всем', 'мира', 'стабильного', 'дохода']


In [17]:
def preprocess_doc(doc_text):
    words_list = remove_stopwords(nltk.word_tokenize(doc_text))
# пока без леммизации
    # lemmas_list = lemmatize_words(words_list)
    lemmas_list = words_list
    return " ".join(lemmas_list)

In [18]:
def preprocess_lemma_doc(doc_text):
    words_list = remove_stopwords(nltk.word_tokenize(doc_text))
    lemmas_list = lemmatize_words(words_list)
    return " ".join(lemmas_list)

In [19]:
lemmas_doc = preprocess_doc(seq_list[1100])
lemmas_doc

'//s15.postimg.org/dv7fam3ln/a2110ac82294.jpg всем привет недавно одна девочка звала crp.center освободилась 1200 долларов думаю эххх ... вложила криптовалюту верила проект знаю почему слава богу интуиция подвела говорит 1000 долларов сняла туда 000 слава богу представьте сколько людей потеряло печально видео обращение виталий говорит будут работать обычно сказки вкладывайте //www.youtube.com/watch time_continue=393 v=g9v3ix73moy всем мира стабильного дохода'

In [21]:
from nltk import FreqDist
fd = FreqDist(nltk.word_tokenize(lemmas_doc))
fd.most_common(50)

[('говорит', 2),
 ('богу', 2),
 ('слава', 2),
 ('долларов', 2),
 ('всем', 2),
 ('дохода', 1),
 ('1000', 1),
 ('//s15.postimg.org/dv7fam3ln/a2110ac82294.jpg', 1),
 ('почему', 1),
 ('звала', 1),
 ('обращение', 1),
 ('будут', 1),
 ('представьте', 1),
 ('000', 1),
 ('...', 1),
 ('знаю', 1),
 ('освободилась', 1),
 ('сколько', 1),
 ('девочка', 1),
 ('crp.center', 1),
 ('мира', 1),
 ('интуиция', 1),
 ('вкладывайте', 1),
 ('вложила', 1),
 ('сняла', 1),
 ('подвела', 1),
 ('1200', 1),
 ('недавно', 1),
 ('проект', 1),
 ('печально', 1),
 ('одна', 1),
 ('time_continue=393', 1),
 ('//www.youtube.com/watch', 1),
 ('v=g9v3ix73moy', 1),
 ('видео', 1),
 ('верила', 1),
 ('сказки', 1),
 ('виталий', 1),
 ('криптовалюту', 1),
 ('думаю', 1),
 ('стабильного', 1),
 ('работать', 1),
 ('эххх', 1),
 ('потеряло', 1),
 ('привет', 1),
 ('людей', 1),
 ('туда', 1),
 ('обычно', 1)]

In [22]:
fout = codecs.open('datasets/golos_io_topics-res-lines.txt', encoding='utf-8', mode='w+')
for item in seq_list:
    if item is not None:
        processed_doc = preprocess_doc(item)
        print(processed_doc, file=fout, sep="\n")
fout.close()

In [23]:
fout = codecs.open('datasets/golos_io_topics-res-lemmas.txt', encoding='utf-8', mode='w+')
for item in seq_list:
    if item is not None:
        processed_lemma_doc = preprocess_lemma_doc(item)
        print(processed_lemma_doc, file=fout, sep="\n")
fout.close()

In [27]:
text = "люди употребляют различные препараты делятся своими впечатлениями . акцент ноотропы , продление жизни . всему этому процессу хватает организованности . собственно этим займемся . прекрасный план . шаг 1. личный медицинский кабинет . должны результаты ваших анализов . сдавали . терапии , которые осуществляли . самое важно , должны результаты диагностики старения . плюс , , описание интервенций против старения . биомаркеры скорости старения далеки совершенств"

In [28]:
from pymystem3 import Mystem
m = Mystem()
lemmas = m.analyze(text)
// print(lemmas)

SyntaxError: invalid syntax (<ipython-input-28-80ffdd9fda40>, line 4)

In [ ]:
lemmas[0]['analysis'][0]['gr'].split(",")[0]